In [ ]:
import pandas as pd
import os

In [ ]:
df = pd.read_csv('/Users/stevie/repos/lingo_kit_data/dataframes/category_data.csv')
len(df), df.columns

In [ ]:
for base in df['english_base_term'].unique():
    subset = df[df['english_base_term'] == base]
    found = False
    for i, row in subset.iterrows():
        if 'to '+row['english_base_term'] == row['english_term']:
            found = True
    if found:
        print(base)
        df.loc[subset.index, 'english_base_term'] = 'to ' + base

In [ ]:
df['both_base_terms'] = df['english_base_term'] + ' | ' + df['italian_base_term']

In [ ]:
df['both_base_terms'].nunique()

In [ ]:
df['is_base'] = 'unknown'
count_dict = {}
for both_base in df['both_base_terms'].unique():
    subset = df[df['both_base_terms'] == both_base]
    if len(subset) == 1:
        df.loc[subset.index, 'is_base'] = True
        continue
    
    if len(subset) == 2:
        masc_mask = subset[subset['english_term'].str.contains('masculine')]
        fem_mask = subset[subset['english_term'].str.contains('feminine')]
        if len(masc_mask) == 1 and len(fem_mask) == 1:
            df.loc[masc_mask.index, 'is_base'] = True
            df.loc[fem_mask.index, 'is_base'] = False
            continue

    if len(subset) == 2:
        item1 = subset.iloc[0]
        item2 = subset.iloc[1]
        eng1 = item1['english_term']
        eng2 = item2['english_term']
        if eng1.endswith('(plural)') and eng1 + ' (plural)' == eng2:
            df.loc[subset.index, 'is_base'] = [False, True]
            continue
        if eng2.endswith('(plural)') and eng2 + ' (plural)' == eng1:
            df.loc[subset.index, 'is_base'] = [True, False]
    
    if len(subset) == 4:
        masc_mask = subset[subset['english_term'].apply(lambda x: x.endswith('(masculine)'))]
        fem_mask = subset[subset['english_term'].apply(lambda x: x.endswith('(feminine)'))]
        masc_plural_mask = subset[subset['english_term'].apply(lambda x: x.endswith('(masculine) (plural)'))]
        fem_plural_mask = subset[subset['english_term'].apply(lambda x: x.endswith('(feminine) (plural)'))]
        if (len(masc_mask) == 1 and len(fem_mask) == 1 and
            len(masc_plural_mask) == 1 and len(fem_plural_mask) == 1):
            df.loc[masc_mask.index, 'is_base'] = True
            df.loc[fem_mask.index, 'is_base'] = False
            df.loc[masc_plural_mask.index, 'is_base'] = False
            df.loc[fem_plural_mask.index, 'is_base'] = False
            continue
    
    english_base_term, italian_base_term = both_base.split(' | ')
    mask = (subset['english_term'] == english_base_term) & (subset['italian_term'] == italian_base_term)
    n = mask.sum()
    if n not in count_dict:
        count_dict[n] = 0
    count_dict[n] += 1
    if n == 1:
        df.loc[mask.index, 'is_base'] = mask
count_dict

In [ ]:
print((df['is_base'] == True).sum())
print((df['is_base'] == False).sum())

In [ ]:
df.sort_values('both_base_terms', inplace=True)

In [ ]:
df.drop(columns=['both_base_terms'], inplace=True)

In [ ]:
df.to_csv('/Users/stevie/repos/lingo_kit_data/dataframes/category_data.csv', index=False)